# Utilisation API Pôle emploi
Source : https://github.com/etiennekintzler/api-offres-emploi

In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.exceptions import HTTPError

import datetime
import time
import re
import pandas as pd 

In [2]:
client_id="PAR_textmining_6361ebc3d9de749cdfb72158572fa975a0ba30e97e4e570957f187132bb2b361"
client_secret="d77c395663a600cb89716cd8a6c02ab9b2d6920fbdc520791bcff93578b2f093"

In [3]:
client_id="PAR_test_085218c044f273f3c6642fcf3a5019ec4c4f6f5c0388dc65141609db5ddf2b12"
client_secret="b903c0540ecb05b669cf5b1fe8d640ee42e46a1b340841a00d7e0656d41abaa1"

In [4]:
def get_token(client_id,client_secret):
    data = dict(
                grant_type="client_credentials",
                client_id=client_id,
                client_secret=client_secret,
                scope="api_offresdemploiv2 o2dsoffre application_{}".format(client_id),
            )

    headers = {"content-type": "application/x-www-form-urlencoded"}
    params = dict(realm="/partenaire")
    current_time = datetime.datetime.today()
    r = requests.post(
        url="https://entreprise.pole-emploi.fr/connexion/oauth2/access_token",
        headers=headers,
        data=data,
        params=params,
        timeout=60
            )
    try:
        r.raise_for_status()
    except HTTPError as error:
        if r.status_code == 400:
            complete_message = str(error) + "\n" + str(r.json())
            raise HTTPError(complete_message)
        else:
            raise error
    else:
        token = r.json()
        token["expires_at"] = current_time + datetime.timedelta(
            seconds=token["expires_in"]
        )
        
        return token

In [5]:
token=get_token(client_id,client_secret)
token

{'access_token': '4E1LqUq-gl7YJAGgMw4QrIRBJkE',
 'scope': 'application_PAR_test_085218c044f273f3c6642fcf3a5019ec4c4f6f5c0388dc65141609db5ddf2b12 api_offresdemploiv2 o2dsoffre',
 'token_type': 'Bearer',
 'expires_in': 1499,
 'expires_at': datetime.datetime(2024, 1, 6, 20, 37, 46, 888046)}

In [6]:
def get_hearders(token):
    headers = {
            "Authorization": "Bearer {}".format(token["access_token"])
        }
    return headers


In [7]:
# Valeurs constantes

OFFRES_DEMPLOI_V2_BASE = "https://api.emploi-store.fr/partenaire/offresdemploi/v2"
REFERENTIEL_ENDPOINT = "{}/referentiel".format(OFFRES_DEMPLOI_V2_BASE)
SEARCH_ENDPOINT = "{}/offres/search".format(OFFRES_DEMPLOI_V2_BASE)

In [8]:
# 'Réferentiel' available: domaine, appellations (domaines professionnelles ROME), metiers, themes, continents,
#         pays, regions, departements , communes , secteursActivites, naturesContrats,  typesContrats, niveauxFormations,
#         permis, langues

def referentiel(ref):
    
    r = requests.Session().get(
                url="{}/{}".format(REFERENTIEL_ENDPOINT, ref),
                params= dict(realm="/partenaire"),
                headers=get_hearders()
            )

    try:
        r.raise_for_status()
    except Exception as e:
        raise e
    else:
        return r.json()

In [9]:
def search(params=None):
    print('Requête avec les params {}'.format(params))
    r = requests.Session().get(
            url=SEARCH_ENDPOINT,
            params=params,
            headers=get_hearders(token)
        )

    silent_http_errors=False
    try:
        r.raise_for_status()
    except HTTPError as error:
        if r.status_code == 400:
            complete_message = str(error) + "\n" + r.json()["message"]
            if silent_http_errors:
                print(complete_message)
            else:
                raise HTTPError(complete_message)
        else:
            if silent_http_errors:
                print(str(error))
            else:
                raise error
    else:
        found_range = re.search(
            pattern="offres (?P<first_index>\d+)-(?P<last_index>\d+)/(?P<max_results>\d+)",
            string=r.headers["Content-Range"],
        ).groupdict()
        out = r.json()
        out.update({"Content-Range": found_range})
        return out

# Exemple

In [10]:
start_dt = datetime.datetime(2020, 3, 1, 12, 30)
end_dt = datetime.datetime.today()
params = {

    "motsCles": "data science",
    'minCreationDate': start_dt.strftime("%Y-%m-%dT%H:%M:%SZ"),
    'maxCreationDate': end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
}

ex_search=search(params)


Requête avec les params {'motsCles': 'data science', 'minCreationDate': '2020-03-01T12:30:00Z', 'maxCreationDate': '2024-01-06T20:12:48Z'}


In [11]:
ex_search

{'resultats': [{'id': '166MPJT',
   'intitule': 'Analyste en Business Intelligence et Data Science (H/F)',
   'description': "Aussitôt Bon, c'est la nouvelle manière de manger sainement et rapidement. Une box toutes les semaines avec des produits frais, locaux et sains. Aussitôt Bon s'adresse à ceux qui n'ont pas le temps de cuisiner, mais qui aiment bien manger.\nNous avons fondé l'entreprise, il y a maintenant 2 ans. Et elle est en pleine croissance ! En deux années seulement Aussitôt Bon s'est imposé dans le secteur de la FoodTech. Une croissance à deux chiffres tous les mois !\n\nNotre seul mantra: régaler un maximum nos clients, sans concession\n\nNous recherchons un Analyste en Business Intelligence et Data Science talentueux en CDI pour rejoindre notre équipe dynamique. Le candidat retenu jouera un rôle clé dans la création de reportings analytics réguliers, l'amélioration des KPIs, la conception de tableaux de bord interactifs, et la production de rapports détaillés pour souten

In [12]:
import sqlite3

In [13]:
conn = sqlite3.connect('job_mining.db')

In [14]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS LieuTravail_Dimension (
    id TEXT PRIMARY KEY,
    libelle TEXT,
    latitude REAL,
    longitude REAL,
    codePostal TEXT,
    commune TEXT
)
''')



In [15]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Entreprise_Dimension (
    id TEXT PRIMARY KEY,
    nom TEXT,
    adaptee INTEGER
)
''')


In [16]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Contrat_Dimension (
    id TEXT PRIMARY KEY,
    typeContrat TEXT,
    typeContratLibelle TEXT,
    natureContrat TEXT,
    dureeTravailLibelle TEXT,
    dureeTravailLibelleConverti TEXT,
    alternance INTEGER
)
''')

In [17]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS OrigineOffre_Dimension (
        id TEXT PRIMARY KEY,
        origine TEXT,
        urlOrigine TEXT
    )
''')

In [18]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Qualification_Dimension (
        id TEXT PRIMARY KEY,
        qualification TEXT
    )
''')

In [19]:
# Création de la table de faits (OffresEmploi_Faits)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS OffresEmploi_Faits (
        id TEXT PRIMARY KEY,
        dateCreation TEXT,
        dateActualisation TEXT,
        nombrePostes INTEGER,
        salaireLibelle TEXT,
        lieuTravailId TEXT REFERENCES LieuTravail_Dimension(id),
        entrepriseId TEXT REFERENCES Entreprise_Dimension(id),
        contratId TEXT REFERENCES Contrat_Dimension(id),
        qualificationId TEXT REFERENCES Qualification_Dimension(id),
        origineOffreId TEXT REFERENCES OrigineOffre_Dimension(id)
    )
''')

In [20]:
# Commit et fermeture de la connexion
conn.commit()
conn.close()

## Chargement table  --- LieuTravail_Dimension

In [21]:
data = {
    'id': [],
    'libelle': [],
    'latitude': [],
    'longitude': [],
    'codePostal': [],
    'commune': []
}

for job_posting in ex_search.get('resultats', []):
    lieu_travail = job_posting.get('lieuTravail', {})
    
    data['id'].append(lieu_travail.get('libelle', ''))
    data['libelle'].append(lieu_travail.get('libelle', ''))
    data['latitude'].append(lieu_travail.get('latitude', ''))
    data['longitude'].append(lieu_travail.get('longitude', ''))
    data['codePostal'].append(lieu_travail.get('codePostal', ''))
    data['commune'].append(lieu_travail.get('commune', ''))

# Create a DataFrame
df_lieu = pd.DataFrame(data)
df_lieu = df_lieu.drop_duplicates(subset=['libelle'])
df_lieu = df_lieu.reset_index(drop=True)
df_lieu['id'] = df_lieu.index + 1
df_lieu.head()

,id,libelle,latitude,longitude,codePostal,commune
0,1,75 - PARIS 01,48.859000,2.347000,75001,75101
1,2,59 - LILLE,50.630992,3.045433,59000,59350
2,3,75 - PARIS 02,48.867745,2.341157,75002,75102
3,4,93 - AUBERVILLIERS,48.912170,2.384455,93300,93001
4,5,13 - AIX EN PROVENCE,43.541369,5.406124,13080,13001


## Chargement de la table --- Entreprise_Dimension

In [22]:
data_entreprises = {
    'id': [],
    'nom': [],
    'adaptee': []
}

for job_posting in ex_search.get('resultats', []):
    entreprise = job_posting.get('entreprise', {})
    
    data_entreprises['id'].append(entreprise.get('nom', ''))
    data_entreprises['nom'].append(entreprise.get('nom', ''))
    data_entreprises['adaptee'].append(entreprise.get('entrepriseAdaptee', ''))

# Create a DataFrame for entreprises
df_entreprises = pd.DataFrame(data_entreprises)
df_entreprises = df_entreprises.drop_duplicates(subset=['nom'])
df_entreprises = df_entreprises.reset_index(drop=True)
df_entreprises['id'] = df_entreprises.index + 1
df_entreprises.head()


,id,nom,adaptee
0,1,JOURNEY,False
1,2,VEKIA SAS,False
2,3,LUNALOGIC,False
3,4,Saint Gobain France,False
4,5,,False


## Chargement de la table --- Contrat_Dimension

In [23]:
data_contrat = {
    'id': [],
    'typeContrat': [],
    'typeContratLibelle': [],
    'natureContrat': [],
    'dureeTravailLibelle': [],
    'dureeTravailLibelleConverti': [],
    'alternance': []
}

for job_posting in ex_search.get('resultats', []):
    contrat = job_posting
    
    data_contrat['id'].append(contrat.get('id', ''))
    data_contrat['typeContrat'].append(contrat.get('typeContrat', ''))
    data_contrat['typeContratLibelle'].append(contrat.get('typeContratLibelle', ''))
    data_contrat['natureContrat'].append(contrat.get('natureContrat', ''))
    data_contrat['dureeTravailLibelle'].append(contrat.get('dureeTravailLibelle', ''))
    data_contrat['dureeTravailLibelleConverti'].append(contrat.get('dureeTravailLibelleConverti', ''))
    data_contrat['alternance'].append(contrat.get('alternance', ''))

# Create a DataFrame for contrat
df_contrat = pd.DataFrame(data_contrat)
df_contrat['id'] = df_contrat.index + 1
df_contrat.head()


,id,typeContrat,typeContratLibelle,natureContrat,dureeTravailLibelle,dureeTravailLibelleConverti,alternance
0,1,CDI,Contrat à durée indéterminée,Contrat travail,39H Travail en journée,Temps plein,False
1,2,CDD,Contrat à durée déterminée - 12 Mois,Cont. professionnalisation,35H Travail en journée,Temps plein,True
2,3,CDI,Contrat à durée indéterminée,Contrat travail,35H Travail en journée,Temps plein,False
3,4,CDI,Contrat à durée indéterminée,Contrat travail,,,False
4,5,CDI,Contrat à durée indéterminée,Contrat travail,,,False


## Chargement de la table --- OrigineOffre_Dimension

In [24]:
data_origine = {
    'id': [],
    'origine': [],
    'urlOrigine': []
}

for job_posting in ex_search.get('resultats', []):
    origine_offre = job_posting.get('origineOffre', {})
    
    data_origine['id'].append(origine_offre.get('id', ''))
    data_origine['origine'].append(origine_offre.get('origine', ''))
    data_origine['urlOrigine'].append(origine_offre.get('urlOrigine', ''))

# Create a DataFrame for origine
df_origine = pd.DataFrame(data_origine)
df_origine['id'] = df_origine.index + 1
df_origine

,id,origine,urlOrigine
0,1,1,https://candidat.pole-emploi.fr/offres/recherc...
1,2,1,https://candidat.pole-emploi.fr/offres/recherc...
2,3,1,https://candidat.pole-emploi.fr/offres/recherc...
3,4,2,https://candidat.pole-emploi.fr/offres/recherc...
4,5,2,https://candidat.pole-emploi.fr/offres/recherc...
5,6,2,https://candidat.pole-emploi.fr/offres/recherc...
6,7,2,https://candidat.pole-emploi.fr/offres/recherc...
7,8,2,https://candidat.pole-emploi.fr/offres/recherc...
8,9,2,https://candidat.pole-emploi.fr/offres/recherc...


## Chargement de la table --- Qualification_Dimension

In [25]:
data_qualification = {
    'id': [],
    'qualification': []
}
for job_posting in ex_search.get('resultats', []):
    data_qualification['id'].append(job_posting.get('id', ''))
    data_qualification['qualification'].append(job_posting.get('qualificationLibelle', ''))

# Create a DataFrame for qualification
df_qualification = pd.DataFrame(data_qualification)
df_qualification = df_qualification.drop_duplicates(subset=['qualification'])
df_qualification = df_qualification.reset_index(drop=True)
df_qualification['id'] = df_qualification.index + 1
df_qualification


,id,qualification
0,1,Agent de maîtrise
1,2,Technicien
2,3,Cadre
3,4,


## Chargement de la table --- OffresEmploi_Faits

In [26]:
import sqlite3
import pandas as pd

# Extracting data for all job postings
data = {
    'id': [],
    'dateCreation': [],
    'dateActualisation': [],
    'nombrePostes': [],
    'salaireLibelle': [],
    'lieuTravailId': [],
    'entrepriseId': [],
    'contratId': [],
    'qualificationId': [],
    'origineOffreId': []
}

for job_posting in ex_search.get('resultats', []):
    data['id'].append(job_posting.get('id', ''))
    data['dateCreation'].append(job_posting.get('dateCreation', ''))
    data['dateActualisation'].append(job_posting.get('dateActualisation', ''))
    data['nombrePostes'].append(job_posting.get('nombrePostes', ''))
    data['salaireLibelle'].append(job_posting.get('salaire', {}).get('libelle', ''))
    data['lieuTravailId'].append(job_posting.get('lieuTravail', {}).get('libelle', ''))
    data['entrepriseId'].append(job_posting.get('entreprise', {}).get('nom', ''))
    data['contratId'].append(job_posting.get('typeContratLibelle', ''))
    data['qualificationId'].append(job_posting.get('qualificationLibelle', ''))
    data['origineOffreId'].append(job_posting.get('origineOffre', {}).get('origine', ''))

# Create a DataFrame
df_offreF = pd.DataFrame(data)


In [27]:
df_lieu_selection = df_lieu[['id', 'libelle']]
df_entreprises_selection = df_entreprises[['id', 'nom']]
df_contrat_selection = df_contrat[['id', 'typeContratLibelle']]
df_origine_selection = df_origine[['id', 'origine']]
df_qualification_selection = df_qualification[['id', 'qualification']]

#LIEU
df_offreF = pd.merge(df_offreF, df_lieu_selection, how='left', left_on='lieuTravailId', right_on='libelle')
df_offreF = df_offreF.drop(columns=['lieuTravailId','libelle'])
df_offreF = df_offreF.rename(columns={'id_y': 'lieuTravailId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#ENTREPRISE
df_offreF = pd.merge(df_offreF, df_entreprises_selection, how='left', left_on='entrepriseId', right_on='nom')
df_offreF = df_offreF.drop(columns=['entrepriseId','nom'])
df_offreF = df_offreF.rename(columns={'id_y': 'entrepriseId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#CONTRAT
df_offreF = pd.merge(df_offreF, df_contrat_selection, how='left', left_on='contratId', right_on='typeContratLibelle')
df_offreF = df_offreF.drop(columns=['contratId','typeContratLibelle'])
df_offreF = df_offreF.rename(columns={'id_y': 'contratId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#ORIGINE
df_offreF['origineOffreId'] = df_offreF['origineOffreId'].astype(str)
df_offreF = pd.merge(df_offreF, df_origine_selection, how='left', left_on='origineOffreId', right_on='origine')
df_offreF = df_offreF.drop(columns=['origineOffreId','origine'])
df_offreF = df_offreF.rename(columns={'id_y': 'origineOffreId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#QUALIFICATION
df_offreF = pd.merge(df_offreF, df_qualification_selection, how='left', left_on='qualificationId', right_on='qualification')
df_offreF = df_offreF.drop(columns=['qualificationId','qualification'])
df_offreF = df_offreF.rename(columns={'id_y': 'qualificationId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

# Liste des colonnes dans le nouvel ordre souhaité
nouvel_ordre_colonnes = [
    'id',
    'dateCreation',
    'dateActualisation',
    'nombrePostes',
    'salaireLibelle',
    'lieuTravailId',
    'entrepriseId',
    'contratId',
    'qualificationId',
    'origineOffreId'
]
df_offreF = df_offreF[nouvel_ordre_colonnes]

In [28]:
df_offreF.head(1)

,id,dateCreation,dateActualisation,nombrePostes,salaireLibelle,lieuTravailId,entrepriseId,contratId,qualificationId,origineOffreId
0,166MPJT,2023-12-18T21:47:48.000Z,2024-01-06T00:20:45.000Z,1,"Annuel de 32000,00 Euros à 35000,00 Euros sur ...",1,1,1,1,1


## CHARGEMENT DES TABLES

In [29]:
import sqlite3
import pandas as pd
# Connexion à la base de données SQLite
conn = sqlite3.connect('job_mining.db')

# Chargement des tables
df_lieu.to_sql('LieuTravail_Dimension', conn, if_exists='replace', index=False)
df_entreprises.to_sql('Entreprise_Dimension', conn, if_exists='replace', index=False)
df_contrat.to_sql('Contrat_Dimension', conn, if_exists='replace', index=False)
df_origine.to_sql('OrigineOffre_Dimension', conn, if_exists='replace', index=False)
df_qualification.to_sql('Qualification_Dimension', conn, if_exists='replace', index=False)
df_offreF.to_sql('OffresEmploi_Faits', conn, if_exists='replace', index=False)

# Fermer la connexion
conn.close()
